In [16]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from classic_clustering import ClassicClustering
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv('Data.csv',sep='|',encoding='utf8')
textos = list(df['textos'])

In [5]:
%time
cc = ClassicClustering()
cc.define_stop_words()
cc.textos_tratados = textos
cc.stem()
base_tfidf = cc.vec_tfidf()

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 13.6 µs
Comecou a fazer o stemming.
Tempo para fazer o stemming: 197.535870552063



<1800x52231 sparse matrix of type '<class 'numpy.float64'>'
	with 826465 stored elements in Compressed Sparse Row format>

In [8]:
n_dims = 600
X = cc.SVD(base_tfidf, n_dims)

Começou a redução de dimensionalidade.
Número de dimensões de entrada: 52231
600 dimensões explicam 0.8339714958250612 da variância.
Tempo para fazer a redução de dimensionalidade: 37.13816833496094



In [12]:
macrotema_por_norma = list(df['macrotemas'])
macrotemas = list(dict.fromkeys(macrotema_por_norma))

di = dict.fromkeys(macrotema_por_norma)
i=0
for macrotema in macrotemas:
    di[macrotema] = i
    i+=1

y = np.zeros(len(macrotema_por_norma))
i=0
for m in macrotema_por_norma:
    y[i] = di[m]
    i+=1

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
models = [LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'),
          RandomForestClassifier(n_estimators=100, max_depth=4,random_state=0),
          MultinomialNB()]

In [18]:
for model in models:
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))

0.7416666666666667
0.4638888888888889
0.42777777777777776
